In [ ]:
##Longhand - M. Cook - 2022
##Takes text corpus and returns immersive visualization
##https://github.com/Cook4986/Longhand

In [ ]:
#Takes Bag-of-Words and returns json dump of common nouns, Sketchfab models/uids, and relative percentage of occurance
import spacy
from collections import Counter
import json
import requests
from requests.exceptions import RequestException
import time

#select language model (https://spacy.io/models)
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 100000000
stopwords = nlp.Defaults.stop_words

#I/O
BoW = "..."#plaintext bag-of-words input
output = "..." #See: "Objects" declaration, below, for structure

#declarations
model_size = 100000 #face count
SKETCHFAB_API_URL = "https://api.sketchfab.com/v3/search?type=models&count=24&max_face_count=" + str(model_size) #note count parameter
API_TOKEN = '...' #keep private
results = 50 #target number of models, potentially limited by NER outputs
slug = ["categories=food-drink"]#limits search
start = time.time()

#data structures
nouns = [] #nouns in Bag-of-Words
entities = [] #named entities
freqs = [] # noun appearance frequencies
objects = {} # key = common nouns; value(s) = [relative percentage of total objects, UID, model name, URL]

#parse Bag-of-Words parts-of-speech with SpaCy
with open(BoW, encoding="utf-8") as file:
    print("Tokenizing text...")
    print("\n")
    iliad = file.read()
document = nlp(iliad)

#collate nouns or named entities ("results") in corpus, by frequency
for token in document:
    if (token not in stopwords) & (token.pos_ == 'NOUN'): 
            nouns.append(token.text)         
labelsEN = ["PRODUCT"]
#["PERSON",PRODUCT","EVENT","FAC","WORK_OF_ART","LOC","NORP","GPE","ORG"] 
for entity in document.ents:
    for label in labelsEN:
        if (token not in stopwords) & (entity.label_ == label):
            entities.append(entity.text)
word_freq = Counter(nouns) #update for nouns or entities
common = word_freq.most_common(results)

#Sketchfab API payload function 
##From https://sketchfab.com/developers/data-api/v3/python#example-python-model
def _get_request_payload(*, data=None, files=None, json_payload=False):
    """Helper method that returns the authentication token and proper content type depending on
    whether or not we use JSON payload."""
    data = data or {}
    files = files or {}
    headers = {'Authorization': 'Token {}'.format(API_TOKEN)}
    if json_payload:
        headers.update({'Content-Type': 'application/json'})
        data = json.dumps(data)
    return {'data': data, 'files': files, 'headers': headers}

#query sketchfabs with tokens and compile object dictionary with results
for word in common:
    key = str(word[0])
    print("Searching: " + key)
    print("\n")
    query = ("&q="+(key)+"&"+ slug[0] + "&downloadable=true") #granular search, see: "slug" declaration, above
    #query = ("&q="+(key)+"&downloadable=true")
    search_endpoint = f'{SKETCHFAB_API_URL + query}'
    payload = _get_request_payload() 
    response = requests.get(search_endpoint, **payload)
    data = response.json()
    for item in range(len(data['results'])):
        url = (data['results'][item]['uri'])
        uid = (data['results'][item]['uid'])
        name = (str((data['results'][item]['name'])))
        size = int(data['results'][item]['faceCount'])
        if (key.lower() in name.lower()) & (key not in objects):
            freqs.append(word[1])
            objects[word[0]] = [word[1]]
            objects[key] += [name, uid, url, size]
print("\n")      

#write object dictionary to disk 
with open(output, 'w') as file:
    file.write(json.dumps(objects)) 
file.close()

#print hits and relative percentages in corpus
Sum = sum(freqs)
for key,value in objects.items():
    print("Model located for '" + key + "':")
    print(value[1].center(24))
    flowt = (value[0] / Sum) * 100
    percentage = round(flowt, 2)
    print("Represents " + str(percentage) + "% of models identified.")
    if value[4] > 10000:
                print("Warning: Model size exceeds 10000 faces " + "(" + (str(size)) + " faces)")
    print("\n")

print("\n")
print(str(len(objects)) + " most suitable models (of "+ str(results) + ") located on Sketchfab written to disk")
print("\n")

#terminate program
end = time.time()
print(str(end - start) + " seconds elapsed" )
print("\n")
print("have a nice day")


In [ ]:
##Launches Blender from terminal and initiates model download script
!/Applications/Blender.app/Contents/MacOS/Blender --python .../Longhand_downloader.py


In [ ]:
#Notes/To-Do:

##Add "working directory", across scripts, to automate declarations
##lemmatize tokens?
##Mesh collision detection
##user-selected working directory + "name your output" user input (for objects file)
#u#ser-selected "nouns or entities"
##parse entire directory of files (glob)
##GUI: "Select Bag-of-Words", Select working directory", "How many objects?", Nouns or Entities", "Entity Categories"
##Print BoW file size
##spell-check?